In [1]:
# !wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh

# !chmod +x Anaconda3-5.1.0-Linux-x86_64.sh

# !bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local

In [2]:
# !conda update -n base -c defaults conda -y

In [3]:
# !git clone https://github.com/alibaba/EasyNLP.git
# %cd EasyNLP/diffusion/FreePromptEditing/

In [4]:
# !conda env create -f environment.yaml

In [5]:
# !source activate FPE

In [6]:
# !pip install diffusers
# !pip install einops

In [7]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from diffusers import DDIMScheduler

from Freeprompt.diffuser_utils import FreePromptPipeline
from Freeprompt.freeprompt_utils import register_attention_control_new
from torchvision.utils import save_image
from torchvision.io import read_image
from Freeprompt.freeprompt import SelfAttentionControlEdit,AttentionStore


#### Model Construction

In [8]:
# Note that you may add your Hugging Face token to get access to the models
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_path = "runwayml/stable-diffusion-v1-5"
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = FreePromptPipeline.from_pretrained(model_path, scheduler=scheduler).to(device)
device

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

/home/novikova/opt/izar1/cross/lib/python3.10/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:106: FutureWarning: The configuration file of this scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.14.0",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "set_alpha_to_one": false,
  "steps_offset": 0,
  "trained_betas": null
}
 is outdated. `steps_offset` should be set to 1 instead of 0. Please make sure to update the config accordingly as leaving `steps_offset` might led to incorrect results in future versions. If you have downloaded this checkpoint from the Hugging Face Hub, it would be very nice if you could open a Pull request for the `scheduler/scheduler_config.json` file
  deprecate("steps_offset!=1", "1.0.0", deprecation_message, standard_warn=False)


device(type='cuda')

#### Real editing

In [9]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
from PIL import Image

# Note that you may add your Hugging Face token to get access to the models
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def plot_image(image):
    # Convert image tensor to numpy array
    image_np = TF.to_pil_image(image.cpu().squeeze())
    
    # Plot the image
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()

def load_image(image_path, device):
    image = read_image(image_path)
    image = image[:3].unsqueeze_(0).float() / 127.5 - 1.  # [-1, 1]
    image = F.interpolate(image, (512, 512))
    image = image.to(device)
    return image


# def load_image(image_path, device):
#     # Read the image using PIL
#     image = Image.open(image_path)
    
#     # Convert image to RGB if it's not already
#     if image.mode != 'RGB':
#         image = image.convert('RGB')
    
#     # Resize the image to 512x512
#     image = transforms.Resize((512, 512))(image)
    
#     # Convert PIL Image to Tensor
#     image = TF.to_tensor(image)
    
#     # Normalize the image
#     normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Assuming mean=0.5, std=0.5
#     image = normalize(image)
    
#     # Add batch dimension
#     image = image.unsqueeze(0).to(device)
    
#     return image

self_replace_steps = .8
NUM_DIFFUSION_STEPS = 50

# out_dir = "results"

def inference(SOURCE_IMAGE_PATH, target_prompt, output_name, out_dir = "results"):

    source_image = load_image(image_path, device)
    # plot_image(source_image)
    
    source_prompt = ""
    
    # invert the source image
    start_code, latents_list = pipe.invert(source_image,
                                            source_prompt,
                                            guidance_scale=7.5,
                                            num_inference_steps=50,
                                            return_intermediates=True)
    
    
    latents = torch.randn(start_code.shape, device=device)
    
    prompts = [source_prompt, target_prompt]
    
    start_code = start_code.expand(len(prompts), -1, -1, -1)
    controller = SelfAttentionControlEdit(prompts, NUM_DIFFUSION_STEPS, self_replace_steps=self_replace_steps)
    
    register_attention_control_new(pipe, controller)
    
    # Note: querying the inversion intermediate features latents_list
    # may obtain better reconstruction and editing results
    results = pipe(prompts,
                        latents=start_code,
                        guidance_scale=7.5,
                        ref_intermediate_latents=latents_list)
    
    
    save_image(results[1], os.path.join(out_dir, output_name +'.jpg'))


In [10]:
def read_prompt(file_path, index):
    # Read the prompt corresponding to the given index from the file
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if index < len(lines):
            return lines[index].strip()
        else:
            return "No prompt available"

In [11]:
prompts_all = ['Modify the material to leather', 'leather material', 
               'Change the color to pink', 'pink color', 
               'Add a geometric pattern', 'geometric pattern']

prompts_shirts = ['Transform the material of the shirt to silk', 'Change the material of the shirt to denim', 'denim shirt',
                  'Change the color of the shirt to red', 'red shirt',
                  'Apply a floral pattern to the shirt', 'floral pattern shirt',
                  'Modify the sleeve length of shirt to be long', 'Adjust the sleeves of the shirt to be three-quarter length', 'long sleeves',
                  'Adjust the neckline of the shirts to a V-neck', 'Change the neckline of the shirt to a scoop neck', 'scoop neck neckline'
                 ]

prompts_dress = ['Transform the material of the dress to silk', 'Change the material of the dress to denim', 'denim dress',
                 'Change the color of the dress to red', 'red dress',
                 'Apply a floral pattern to the dress', 'floral pattern dress',
                 'Transform the sleeves of the dress to cap sleeves', 'Change the sleeve length of dress to be long', 'cap sleeves',
                 'Lengthen the hemline of the dress to floor length', 'floor length hemline',
                 'Slim down the waist of the dress for a fitted silhouette', 'fitted silhouette']

prompts_pants = ['Transform the material of the pants to silk', 'Change the material of the pants to leather', 'leather pants',
                 'Change the color of the pants to red', 'red pants',
                 'Apply a floral pattern to the pants', 'floral pattern pants',
                 'Shorten the hemline of the pants to knee length', 'knee length hemline',
                 'Expand the width of the leg of the pants for a wide-leg style', 'wide-leg style',
                 'Slim down the width of the leg of the pants for a tapered look', 
                 'Modify the width of the leg of each pair of pants to be flared']

prompts_skirt = ['Transform the material of the skirt to silk', 'Change the material of the skirt to denim', 'denim skirt',
                 'Change the color of the skirt to red', 'red skirt',
                 'Apply a floral pattern to the skirt', 'floral pattern skirt',
                 'Increase the width of the waist of the skirt for a high-waisted look', 'high-waisted skirt']

In [12]:
prompts_d = {'shirt': prompts_shirts, 'dress': prompts_dress, 'pants': prompts_pants, 'skirt': prompts_skirt}

In [ ]:
folder_path = "../../../fashion_correct"
folder_path_output = "./results_fashion"

images = []

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        # Load the image
        image_path = os.path.join(folder_path, filename)

        image_name = os.path.splitext(filename)[0]
        
        # if not ('skirt' in image_name):
        #     continue

        # if image_name == 'skirt3_p':
        #     continue
        
        print(image_name)

        # prompts = prompts_all
        prompts = prompts_all = []

        for cl_type in prompts_d:
            if cl_type in image_name:
                prompts += prompts_d[cl_type]
                current_output = os.path.join(folder_path_output, cl_type)

        prompts += prompts_all

        for prompt in prompts:
            print(prompt)
            output_filename = f"{image_name}_{prompt}.jpg"
            inference(image_path, prompt, output_filename, out_dir = current_output)

skirt3_p
Transform the material of the skirt to silk


DDIM Sampler: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:13<00:00,  3.84it/s]


Change the material of the skirt to denim


DDIM Sampler: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:13<00:00,  3.82it/s]


denim skirt


...

DDIM Sampler: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:13<00:00,  3.79it/s]


Apply a floral pattern to the skirt


DDIM Sampler: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:13<00:00,  3.76it/s]


floral pattern skirt


DDIM Inversion:  70%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                             | 35/50 [00:05<00:02,  6.71it/s]

In [12]:
# for i in range(1):
#     image_path = os.path.join(folder_path, f"{i}.jpg")

#     # existing_image = Image.open(image_path)
#     prompt = read_prompt(prompts_file, i)
#     print(prompt)

#     inference(image_path, prompt, str(i), out_dir = "results")

In [13]:
# Note that you may add your Hugging Face token to get access to the models
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_path = "runwayml/stable-diffusion-v1-5"
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = FreePromptPipeline.from_pretrained(model_path, scheduler=scheduler).to(device)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [14]:

# Note that you may add your Hugging Face token to get access to the models
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


def load_image(image_path, device):
    image = read_image(image_path)
    image = image[:3].unsqueeze_(0).float() / 127.5 - 1.  # [-1, 1]
    image = F.interpolate(image, (512, 512))
    image = image.to(device)
    return image

self_replace_steps = .8
NUM_DIFFUSION_STEPS = 50

out_dir = "examples/outputs"


SOURCE_IMAGE_PATH = "examples/img/000141.jpg"
source_image = load_image(SOURCE_IMAGE_PATH, device)

source_prompt = ""

# invert the source image
start_code, latents_list = pipe.invert(source_image,
                                        source_prompt,
                                        guidance_scale=7.5,
                                        num_inference_steps=50,
                                        return_intermediates=True)

target_prompt = 'a purple bike'

latents = torch.randn(start_code.shape, device=device)
prompts = [source_prompt, target_prompt]

start_code = start_code.expand(len(prompts), -1, -1, -1)
controller = SelfAttentionControlEdit(prompts, NUM_DIFFUSION_STEPS, self_replace_steps=self_replace_steps)

register_attention_control_new(pipe, controller)

# Note: querying the inversion intermediate features latents_list
# may obtain better reconstruction and editing results
results = pipe(prompts,
                    latents=start_code,
                    guidance_scale=7.5,
                    ref_intermediate_latents=latents_list)


save_image(results[1], os.path.join(out_dir, str(target_prompt)+'.jpg'))


DDIM Inversion:  24%|████████████████████████████████████▏                                                                                                                  | 12/50 [00:01<00:05,  6.92it/s]


KeyboardInterrupt: 

In [ ]:

target_prompt = 'a bike'

latents = torch.randn(start_code.shape, device=device)
prompts = [source_prompt, target_prompt]

start_code = start_code.expand(len(prompts), -1, -1, -1)
controller = SelfAttentionControlEdit(prompts, NUM_DIFFUSION_STEPS, self_replace_steps=self_replace_steps)

register_attention_control_new(pipe, controller)

# Note: querying the inversion intermediate features latents_list
# may obtain better reconstruction and editing results
results = pipe(prompts,
                    latents=start_code,
                    guidance_scale=7.5,
                    ref_intermediate_latents=latents_list)


save_image(results[1], os.path.join(out_dir, str(target_prompt)+'.jpg'))

In [ ]:

SOURCE_IMAGE_PATH = "examples/img/chihuahua.jpg"
source_image = load_image(SOURCE_IMAGE_PATH, device)

source_prompt = ""

# invert the source image
start_code, latents_list = pipe.invert(source_image,
                                        source_prompt,
                                        guidance_scale=7.5,
                                        num_inference_steps=50,
                                        return_intermediates=True)

In [ ]:

target_prompt = 'chihuahua eating burger'

latents = torch.randn(start_code.shape, device=device)
prompts = [source_prompt, target_prompt]

start_code = start_code.expand(len(prompts), -1, -1, -1)
controller = SelfAttentionControlEdit(prompts, NUM_DIFFUSION_STEPS, self_replace_steps=self_replace_steps)

register_attention_control_new(pipe, controller)

# Note: querying the inversion intermediate features latents_list
# may obtain better reconstruction and editing results
results = pipe(prompts,
                    latents=start_code,
                    guidance_scale=7.5,
                    ref_intermediate_latents=latents_list)


save_image(results[1], os.path.join(out_dir, str(target_prompt)+'.jpg'))

In [ ]:

target_prompt = 'cat eating ice cream'

latents = torch.randn(start_code.shape, device=device)
prompts = [source_prompt, target_prompt]

start_code = start_code.expand(len(prompts), -1, -1, -1)
controller = SelfAttentionControlEdit(prompts, NUM_DIFFUSION_STEPS, self_replace_steps=self_replace_steps)

register_attention_control_new(pipe, controller)

# Note: querying the inversion intermediate features latents_list
# may obtain better reconstruction and editing results
results = pipe(prompts,
                    latents=start_code,
                    guidance_scale=7.5,
                    ref_intermediate_latents=latents_list)


save_image(results[1], os.path.join(out_dir, str(target_prompt)+'.jpg'))

In [ ]:

target_prompt = 'cat drinks a milkshake '

latents = torch.randn(start_code.shape, device=device)
prompts = [source_prompt, target_prompt]

start_code = start_code.expand(len(prompts), -1, -1, -1)
controller = SelfAttentionControlEdit(prompts, NUM_DIFFUSION_STEPS, self_replace_steps=self_replace_steps)

register_attention_control_new(pipe, controller)

# Note: querying the inversion intermediate features latents_list
# may obtain better reconstruction and editing results
results = pipe(prompts,
                    latents=start_code,
                    guidance_scale=7.5,
                    ref_intermediate_latents=latents_list)


save_image(results[1], os.path.join(out_dir, str(target_prompt)+'.jpg'))

In [ ]:

target_prompt = 'chihuahua eating burger against a background of mountains'

latents = torch.randn(start_code.shape, device=device)
prompts = [source_prompt, target_prompt]

start_code = start_code.expand(len(prompts), -1, -1, -1)
controller = SelfAttentionControlEdit(prompts, NUM_DIFFUSION_STEPS, self_replace_steps=self_replace_steps)

register_attention_control_new(pipe, controller)

# Note: querying the inversion intermediate features latents_list
# may obtain better reconstruction and editing results
results = pipe(prompts,
                    latents=start_code,
                    guidance_scale=7.5,
                    ref_intermediate_latents=latents_list)


save_image(results[1], os.path.join(out_dir, str(target_prompt)+'.jpg'))

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

images_folder_path = 'examples/img/'

image_files = [f for f in os.listdir(images_folder_path) if os.path.isfile(os.path.join(images_folder_path, f))]

for image_file in image_files:
    image_path = os.path.join(images_folder_path, image_file)

    img = Image.open(image_path)
    plt.imshow(img)
    plt.title(image_file)
    plt.axis('off')  # Hide axis
    plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

images_folder_path = 'examples/outputs/'

image_files = [f for f in os.listdir(images_folder_path) if os.path.isfile(os.path.join(images_folder_path, f))]

for image_file in image_files:
    image_path = os.path.join(images_folder_path, image_file)

    img = Image.open(image_path)
    plt.imshow(img)
    plt.title(image_file)
    plt.axis('off')  # Hide axis
    plt.show()